In [0]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from keras.initializers import Constant 

In [0]:
maxlen = 400
batch_size = 32
embedding_dims = 300
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

In [3]:
import os
!echo '{"username":"imrankhan1386","key":"8708dd3c36c4db8c0dc44b3191edfe49"}' > ~/.kaggle/kaggle.json
!kaggle datasets download -d iarunava/imdb-movie-reviews-dataset # api copied from kaggle

 90% 202M/224M [00:05<00:00, 23.1MB/s]
100% 224M/224M [00:05<00:00, 44.5MB/s]


In [4]:
!unzip /content/imdb-movie-reviews-dataset.zip

Output hidden; open in https://colab.research.google.com to view.

In [0]:
import glob
import os

from random import shuffle

def pre_process_data(filepath):
  positive_path = os.path.join(filepath, 'pos')
  negative_path = os.path.join(filepath, 'neg')
  pos_label = 1
  neg_label = 0
  dataset = []
  
  for filename in glob.glob(os.path.join(positive_path, '*.txt')):
    with open(filename, 'r') as f:
      dataset.append((pos_label, f.read()))

  for filename in glob.glob(os.path.join(negative_path, '*.txt')):
    with open(filename, 'r') as f:
      dataset.append((neg_label, f.read()))

  shuffle(dataset)
  return dataset

dataset = pre_process_data('/content/aclimdb/aclImdb/train') 

In [6]:
dataset[10]

(1,
 'Strangely erotic schlock Gothic horror that will be loved by Hammer House of Horror fans the world over. Appears to be an interesting take on "Beauty and the Beast". It is definitely worth a look and surprisingly well acted when taking into consideration the genre and era.<br /><br />Corsets, castles, rutting horses, rampaging faux fur monsters in the woods, proof that no-one had a Brazilian in the 70\'s, and more bodily fluids than you can shake a stick at, what more could you possibly want?!<br /><br />It certainly brightened my Sunday afternoon!')

In [29]:
len(reviews)

25000

In [10]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2019-10-07 02:55:37--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.236.237
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.236.237|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  46.8MB/s    in 34s     

2019-10-07 02:56:12 (46.1 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [8]:
!pip install gensim
from gensim.models import KeyedVectors

In [0]:
# rm -r /content/aclImdb

In [48]:
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz' # from above
# word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True, limit = 200000) # limitng word embeddings to 200000 rather than 3M.

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
from nltk.tokenize import TreebankWordTokenizer

def tokenize_and_vectorize(dataset):
  tokenizer = TreebankWordTokenizer()
  vectorized_data = []
  expected = []
  for sample in dataset:
    tokens = tokenizer.tokenize(sample[1])
    sample_vecs = []
    for token in tokens:
      try:
        sample_vecs.append(word2vec[token])
      except KeyError:
        pass # No matching Token in Google Word2Vec
      
    vectorized_data.append(sample_vecs)
    
  return vectorized_data
        

In [0]:
def collect_expected(dataset):
  """Peel off the target value from the dataset"""
  expected = []
  for sample in dataset:
    expected.append(sample[0])
  return expected

In [0]:
vectorized_data = tokenize_and_vectorize(dataset)
expected = collect_expected(dataset)

In [0]:
def tokenize_data(dataset):
  tokenizer = TreebankWordTokenizer()
  tokenized_data = []
  for val in dataset:
    tokens = tokenizer.tokenize(val[1])
    tokenized_data.append(tokens)
  return tokenized_data

tokenized_data = tokenize_data(dataset)

In [0]:
tokenized_data

In [0]:
import gensim
# sentences takes tokenized data for each sentence
model = gensim.models.Word2Vec(sentences = tokenized_data, size = embedding_dims, workers = 4, min_count=1)

In [0]:
w = list(model.wv.vocab)

In [94]:
len(w)

150478

In [43]:
model.most_similar('horrible')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('terrible', 0.9246589541435242),
 ('awful', 0.8413890600204468),
 ('lame', 0.7613602876663208),
 ('ridiculous', 0.7564971446990967),
 ('dreadful', 0.7551316618919373),
 ('lousy', 0.7296923398971558),
 ('stupid', 0.7182141542434692),
 ('laughable', 0.714479386806488),
 ('bad', 0.7108624577522278),
 ('atrocious', 0.7038550972938538)]

In [51]:
# Save the model
filename = 'imdb_embeddings_word2vec.txt'
model.wv.save_word2vec_format(filename, binary=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
import os
embeddings_index = {}
f = open(os.path.join('', 'imdb_embeddings_word2vec.txt'), encoding = 'utf-8')
for line in f:
  values = line.split()
  word = values[0]
  coef = np.asarray(values[1:])
  embeddings_index[word] = coef
f.close()

In [54]:
line

'escalates. -0.017746726 -0.008385189 -0.0028210606 0.009340712 0.0031268923 0.0023926129 0.013876623 -0.012875831 -0.015828343 0.0015034447 0.00284569 0.0068189465 0.009507384 0.0062131607 -0.004258878 -0.005317072 -0.013633373 -0.014638999 -0.010144741 0.01217807 -0.007941526 0.0044160546 -0.009045886 0.01006565 -0.017569497 -0.012370431 -0.002480494 0.024911258 0.012819499 -0.010335669 0.014862108 0.020811724 0.0041808183 9.270284e-05 -0.00019301253 0.004650382 0.016773226 0.011657474 0.00601062 0.0055162576 0.0036554 -0.00046878797 0.008701472 -0.009129889 0.011077306 0.0023718525 -0.0028099183 -0.0075567933 0.015227918 0.0137526365 0.0038737208 -0.0033434571 0.005395204 -0.0011371728 0.0038031433 0.00085077295 -0.0046632476 -0.02355355 0.0022487366 0.013337158 0.0023156037 0.002442525 -0.0036348235 0.017651964 0.016983692 0.004849297 -0.004240276 -0.0009120754 -0.0026264694 0.008111358 -0.00403557 0.016067827 0.0058976966 -0.0073245917 0.0031362255 2.0631751e-07 0.0023433794 0.005

In [0]:
embeddings_index

In [56]:
len(values)

301

In [0]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [0]:
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(sample[1] for sample in dataset)
sequences = tokenizer_obj.texts_to_sequences(sample[1] for sample in dataset)
sentiment = [sample[0] for sample in dataset]

In [98]:
len(sequences)

25000

In [0]:
# Pad sequences
word_index = tokenizer_obj.word_index
review_pad = pad_sequences(sequences , maxlen = maxlen)

In [69]:
len(word_index)

88582

In [78]:
review_pad.shape

(25000, 400)

In [99]:
review_pad

array([[   0,    0,    0, ...,   40,   96,  613],
       [   0,    0,    0, ...,    6,    3, 8422],
       [  80,  297, 1670, ...,   15,   11, 2576],
       ...,
       [   0,    0,    0, ...,   43,    4,  155],
       [   0,    0,    0, ...,   43,    4,  454],
       [   0,    0,    0, ...,   10,  383,    9]], dtype=int32)

In [76]:
len(sentiment)

25000

In [0]:
num_words = len(word_index)+1
embedding_matrix = np.zeros((num_words, embedding_dims))

for word,i in word_index.items():
  if i > num_words:
    continue
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector
    

In [103]:
embedding_matrix.shape

(88583, 300)

In [15]:
# This is the whole sentence with each word having 300 dimensions or Topics.
vectorized_data[0]

[array([-0.17285156,  0.27929688,  0.10693359, -0.15820312, -0.08447266,
         0.05908203,  0.04077148,  0.00254822,  0.25976562,  0.18066406,
         0.09765625, -0.08105469, -0.01049805,  0.09814453,  0.00060272,
         0.07080078, -0.015625  , -0.09521484, -0.08105469, -0.02868652,
        -0.03320312,  0.16503906,  0.03979492, -0.03710938,  0.04101562,
        -0.12695312, -0.12890625,  0.12353516,  0.04980469,  0.01257324,
         0.05786133, -0.00830078, -0.02832031, -0.03320312,  0.16113281,
         0.07519531, -0.25976562,  0.08935547,  0.13574219,  0.00460815,
        -0.04418945,  0.02319336, -0.10449219, -0.05151367,  0.08349609,
        -0.02050781, -0.02172852, -0.02734375,  0.16015625,  0.19042969,
        -0.0324707 ,  0.06787109,  0.10302734, -0.25390625,  0.00634766,
         0.20507812,  0.02111816, -0.21679688, -0.02441406,  0.17089844,
        -0.21875   ,  0.10009766, -0.15527344, -0.12597656, -0.03833008,
        -0.05419922,  0.19238281,  0.21777344,  0.1

In [0]:
def pad_trunc(data, maxlen):
  """For a given dataset pad with zero vectors or truncate to maxlen"""
  new_data = [] 
  # Create a vector of 0s the length of our word vectors
  
  zero_vector = []
  for _ in range(len(data[0][0])):
    zero_vector.append(0.0)
  
  for sample in data:
    if len(sample) > maxlen:
      temp = sample[:maxlen]
    elif len(sample) < maxlen:
      temp = sample
      # Append the appropriate number 0 vectors to the list
      additional_elems = maxlen - len(sample)
      for _ in range(additional_elems):
        temp.append(zero_vector)
    else:
      temp = sample
    new_data.append(temp)
  return new_data

In [0]:
x_train = pad_trunc(X_train, maxlen)
x_test = pad_trunc(X_test, maxlen)

In [0]:
# x_train = np.array(x_train)

In [0]:
# Experimenting with just X_train


In [35]:
# type(embedding_layer)

keras.layers.embeddings.Embedding

In [0]:
# xtrain = np.reshape(x_train, (len(x_train), maxlen, embedding_dims))
# ytrain = np.array(y_train)


In [0]:
# xtest = np.reshape(x_test, (len(x_test), maxlen, embedding_dims))
# ytest = np.array(y_test)

In [109]:
print('Build model...')
model = Sequential()
embedding_layer = Embedding(num_words,
                           embedding_dims, 
                           embeddings_initializer = Constant(embedding_matrix),
                           input_length = 400,
                           trainable = False)
model.add(embedding_layer)
model.add(Conv1D(
filters,
kernel_size,
padding='valid',
activation='relu',
strides=1,
input_shape=(maxlen, embedding_dims)))

model.add(GlobalMaxPooling1D())

model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

Build model...



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [110]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [111]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 400, 300)          26574900  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 398, 250)          225250    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 250)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               62750     
_________________________________________________________________
dropout_1 (Dropout)          (None, 250)               0         
_________________________________________________________________
activation_1 (Activation)    (None, 250)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                

In [0]:
# Train Test split
split_point = int(len(review_pad)*.8)

In [113]:
split_point

20000

In [0]:
X_train = review_pad[:split_point]
y_train = sentiment[:split_point]

X_test = review_pad[split_point:]
y_test = sentiment[split_point:]

In [118]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=25, validation_data=(X_test, y_test))

Train on 20000 samples, validate on 5000 samples
Epoch 1/25
20000/20000 [==============================] - 10s 496us/step - loss: 0.3078 - acc: 0.8676 - val_loss: 0.3281 - val_acc: 0.8560
Epoch 2/25
20000/20000 [==============================] - 10s 490us/step - loss: 0.2685 - acc: 0.8885 - val_loss: 0.3024 - val_acc: 0.8670
Epoch 3/25
20000/20000 [==============================] - 10s 489us/step - loss: 0.2397 - acc: 0.8987 - val_loss: 0.3040 - val_acc: 0.8692
Epoch 4/25
20000/20000 [==============================] - 10s 488us/step - loss: 0.2136 - acc: 0.9144 - val_loss: 0.3413 - val_acc: 0.8652
Epoch 5/25
20000/20000 [==============================] - 10s 489us/step - loss: 0.1925 - acc: 0.9229 - val_loss: 0.4074 - val_acc: 0.8534
Epoch 6/25
20000/20000 [==============================] - 10s 493us/step - loss: 0.1545 - acc: 0.9390 - val_loss: 0.3338 - val_acc: 0.8730
Epoch 7/25
20000/20000 [==============================] - 10s 493us/step - loss: 0.1382 - acc: 0.9440 - val_loss: 0.4

In [0]:
model_structure = model.to_json()
with open("cnn_model.json", "w") as json_file:
  json_file.write(model_structure)
model.save_weights("cnn_weights.h5")

In [0]:
# Loading a saved model

from keras.models import model_from_json
with open("cnn_model.json", "r") as json_file:
  json_string = json_file.read()
model = model_from_json(json_string)
model.load_weights('cnn_weights.h5')

In [0]:
sample_1 = "I hate that the dismal weather had me down for so long, when will it break! Ugh, when does happiness return? The sun is blinding\ 
and the puffy clouds are too thin. I can't wait for the weekend."

In [0]:
vec_list = tokenize_and_vectorize([(1, sample_1)])
test_vec_list = pad_trunc(vec_list, maxlen)
test_vec = np.reshape(test_vec_list, (len(test_vec_list), maxlen, embedding_dims))
model.predict(test_vec)

In [0]:
model.predict_classes(test_vec)